In [1]:
!pip3 install kfp


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 119 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.5 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-0.5.1-py3-none-any.whl size=163151 sha256=da5b540ae9834d37659146f0576997ffd8f7a7e2b305e1eb7b2a99dd4745930b
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/26/f9/e3836cb6e6cabd63ef912304e18a852ac29cb870a4a0b85f98
  Created wheel for kfp-server-api: filename=kfp_server_api-0.5.0-py3-none-any.whl size=106319 sha256=84f55948cc254c0f836dffdfd51574a828ae8a503a2ca9198acf7a27ca2aaea7
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/36/4e/bfe2efeeea4f74f04984ebe1d44136202b72191302f4760951
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=3bcfd573a91f5f4c46d23509ac3fee9a0cf351b414e00ed505a8f71d0e6a1141
  Stored in di

In [1]:
import kfp
from kfp import components

In [2]:
COMPONENT_URI = 'https://raw.githubusercontent.com/MavenCode/KubeflowTraining/feature/ctscan-pipeline/Day%202/KubeflowComponentsAndPipeline/Labs/5_chicago_taxi_cab/components/'

chicago_taxi_dataset_op = components.load_component_from_url(f'{COMPONENT_URI}/chicago_taxi_trips/component.yaml')
pandas_transform_csv_op = components.load_component_from_url(f'{COMPONENT_URI}/pandas_transform_df/component.yaml')

train_classifier_op = components.load_component_from_url(f'{COMPONENT_URI}/train_classifier/component.yaml')
train_regression_op = components.load_component_from_url(f'{COMPONENT_URI}/train_regression/component.yaml')
predict_classes_op = components.load_component_from_url(f'{COMPONENT_URI}/predict_classes/component.yaml')
predict_values_op = components.load_component_from_url(f'{COMPONENT_URI}/predict_values/component.yaml')

predict_class_probabilities_op = components.load_component_from_url(
    f'{COMPONENT_URI}/predict_class_probabilities/component.yaml')
export_model_to_AppleCoreML_op = components.load_component_from_url(
    f'{COMPONENT_URI}/export_model_to_AppleCoreML/component.yaml')
export_model_to_ONNX_op = components.load_component_from_url(f'{COMPONENT_URI}/export_model_to_ONNX/component.yaml')

In [16]:
@kfp.dsl.pipeline(
    name="The Chicago Taxi Cab",
    description="The Chicago Taxi Cab Example",
)
def chicago_taxi_pipeline():

    training_data_in_csv = chicago_taxi_dataset_op(
        where='trip_start_timestamp >= "2019-01-01" AND trip_start_timestamp < "2019-02-01"',
        select='tips,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total',
        limit=10000,
    ).output

    training_data_for_classification_in_csv = pandas_transform_csv_op(
        table=training_data_in_csv,
        transform_code='''df.insert(0, "was_tipped", df["tips"] > 0); del df["tips"]''',
    ).output

    evaluation_data_for_regression_in_csv = training_data_in_csv
    evaluation_data_for_classification_in_csv = training_data_for_classification_in_csv

    train_regression_task = train_regression_op(
        training_data="training_data_in_csv",
        loss_function='RMSE',
        label_column=0,
        num_iterations=200,
    )

    regression_model = train_regression_task.outputs['model']


    

kfp.compiler.Compiler().compile(chicago_taxi_pipeline,"chicago_taxi_pipeline.zip")
client = kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888')
# client.list_pipelines()
pipeline_info = client.upload_pipeline('chicago_taxi_pipeline-2.zip',pipeline_name='chicago_taxi_pipeline-2')


In [17]:

my_experiment = client.create_experiment(name='dicom-experiments')
my_run = client.run_pipeline(my_experiment.id, 'my-run1', 'dicom-pipeline-2.zip')